In [ ]:
%load_ext tensorboard

In [ ]:
# This checks if
# (a) the code is running in google colab
# (b) the repository hasn't been cloned yet
import sys
from pathlib import Path
if "google.colab" in sys.modules:
    target_path = Path("/content/license_plate_detection/src/license_plate_extraction")
    if not target_path.exists():
        !git clone https://github.com/cxan96/license_plate_detection /content/license_plate_detection/
    %cd $str(target_path)

In [ ]:
import tensorflow as tf
from tensorflow.keras import losses
import numpy as np
from data_reader import get_image_paths_from_directory, make_dataset_from_image_paths
from preprocessing import bounding_box_in_pixel
from visualization_tools import show_image
import data_augmentation
import settings
import models
import datetime

In [ ]:
tf.test.gpu_device_name()

Read all the available images into a list of paths:

In [ ]:
image_directory_eu = settings.DATA_DIR / "eu_cars+lps"
image_directory_br = settings.DATA_DIR / "br_cars+lps"
image_directory_us = settings.DATA_DIR / "us_cars+lps"
image_directory_ro = settings.DATA_DIR / "ro_cars+lps"
image_directory_russia = settings.DATA_DIR / "cars_russia"

# make a list with all the image paths to use during training
image_paths_train = np.array(
    [
        *get_image_paths_from_directory(image_directory_eu, contains="_car_"),
        *get_image_paths_from_directory(image_directory_br, contains="_car_"),
        *get_image_paths_from_directory(image_directory_us, contains="_car_"),
        *get_image_paths_from_directory(image_directory_ro, contains="_car_"),
        *get_image_paths_from_directory(image_directory_russia),
    ]
)

validation_directory_eu = settings.DATA_DIR / "validation_eu"
validation_directory_ro = settings.DATA_DIR / "validation_ro"

# the image paths used for validation
image_paths_test = np.array(
    [
        *get_image_paths_from_directory(validation_directory_eu, contains="_car_"),
        *get_image_paths_from_directory(validation_directory_ro, contains="_car_")
    ]
)

num_images_train = len(image_paths_train)
num_images_test = len(image_paths_test)

print(f"Number of training images = {num_images_train}")
print(f"Number of test images = {num_images_test}")

Create the training set as well as the test set

In [ ]:
TARGET_IMG_HEIGHT = 400
TARGET_IMG_WIDTH = 400
BATCH_SIZE = 32

dataset_train = make_dataset_from_image_paths(
    image_paths_train,
    target_img_height=TARGET_IMG_HEIGHT,
    target_img_width=TARGET_IMG_WIDTH,
)

# add data augmentation
dataset_train = data_augmentation.horizontal_flip(dataset_train)
# dataset_train = data_augmentation.add_brightness(dataset_train)
# dataset_train = data_augmentation.add_contrast(dataset_train)

# set batch size
dataset_train = dataset_train.shuffle(1024).batch(BATCH_SIZE)

dataset_test = make_dataset_from_image_paths(
    image_paths_test,
    target_img_height=TARGET_IMG_HEIGHT,
    target_img_width=TARGET_IMG_WIDTH,
)
dataset_test = dataset_test.batch(BATCH_SIZE)

Define a model:

In [ ]:
def simple_model(input_height, input_width, num_channels):
    model = tf.keras.Sequential(
        [
            tf.keras.Input(shape=(input_height, input_width, num_channels)),
            tf.keras.layers.experimental.preprocessing.Rescaling(
                1.0 / 255,
            ),
            tf.keras.layers.Conv2D(16, 3, padding="same", activation="relu"),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(4, activation="sigmoid"),
        ]
    )

    return model

model = simple_model(
    input_height=TARGET_IMG_HEIGHT, input_width=TARGET_IMG_WIDTH, num_channels=3
)

model.summary()

Set up the optimizer

In [ ]:
model.compile(optimizer="adam", loss=losses.MAE, metrics=[tf.keras.metrics.MeanAbsoluteError()])

Launch TensorBoard:

In [ ]:
%tensorboard --logdir $settings.LOG_DIR

Fit the model to the training data:

In [ ]:
# add Tensorboard callback
log_dir = settings.LOG_DIR / (
    "training_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1, update_freq="batch"
)

# this makes sure the model is regularly saved to not lose any progress made during training
epochs_per_checkpoint = 5
class ModelSaveCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch > 1 and epoch % epochs_per_checkpoint == 0:
            cur_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            model.save(f"{cur_time}_epoch_{epoch}.tf")
            print("Saved results!")


# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint("checkpoint_epoch_{epoch}.tf")

model.fit(
    dataset_train,
    epochs=1000,
    validation_data=dataset_test,
    callbacks=[tensorboard_callback, ModelSaveCallback()],
)

Evaluate the model:

In [ ]:
model.evaluate(dataset_test)

Show the predictions on the test set

In [ ]:
example_list = list(dataset_test.as_numpy_iterator())

for cur_example in example_list:
    cur_image_batch = cur_example[0]
    cur_prediction_batch = model.predict(cur_image_batch)
    
    for cur_image, cur_prediction in zip(cur_image_batch, cur_prediction_batch):
        print(cur_prediction)
        
        show_image(
            cur_image.astype(int),
            bounding_box=bounding_box_in_pixel(
                cur_prediction,
                img_height=TARGET_IMG_HEIGHT,
                img_width=TARGET_IMG_WIDTH,
            ),
        )